##### Imports

In [ ]:
import sys
from pathlib import Path
import warnings

import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

import sys
# Custom library paths
sys.path.extend(['../', './scr'])

from scr.utils import set_seed
from scr.utils import read_words

set_seed(42)

import torch
import torch.nn as nn

torch.set_float32_matmul_precision('medium')

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from pathlib import Path
import random
from collections import Counter, defaultdict
import pickle
from tqdm import tqdm
from torch.utils.data import Dataset
from scr.feature_engineering import calculate_char_frequencies, calculate_word_frequencies
from scr.utils import read_words, save_words_to_file
from scr.dataset import HangmanDataset, stratified_sample_by_length_and_frequency

# Constants and File Paths
MASK_PROB = 0.8
NGRAM_N = 3
NUM_STRATIFIED_SAMPLES = 100
BATCH_SIZE = 64  # Example batch size, adjust as needed
# base_dataset_dir = Path('data/20k/')
# Define the base directory for the dataset
base_dataset_dir = Path('dataset/20k/')

pkls_dir = base_dataset_dir / 'pkl'
base_dataset_dir.mkdir(parents=True, exist_ok=True)
pkls_dir.mkdir(parents=True, exist_ok=True)

# Read and Shuffle Word List
word_list = read_words('data/20k.txt', limit=1000)
random.shuffle(word_list)

# Splitting Dataset Function
def split_dataset(word_list, train_ratio=0.7, val_ratio=0.15):
    total_words = len(word_list)
    train_size = int(total_words * train_ratio)
    val_size = int(total_words * val_ratio)
    random.shuffle(word_list)
    return word_list[:train_size], word_list[train_size:train_size + val_size], \
        word_list[train_size + val_size:]


# Splitting the word list
train_words, val_words, test_words = split_dataset(word_list)

# Save split datasets to files
save_words_to_file(train_words, base_dataset_dir / 'train_words.txt')
save_words_to_file(val_words, base_dataset_dir / 'val_words.txt')
save_words_to_file(test_words, base_dataset_dir / 'test_words.txt')

# Calculate Frequencies and Max Word Length
word_frequencies = calculate_word_frequencies(train_words)
char_frequency = calculate_char_frequencies(train_words)
max_word_length = max(len(word) for word in train_words)

In [ ]:
import pickle
from pathlib import Path
from scr.dataset import process_word

# Assuming HangmanDataset and other necessary functions are defined

iteration = 0
pkls_dir = pkls_dir # Path("/data_generation_output")  # Ensure this directory exists

while train_words:  # Using train_words for dataset generation
    sampled_words = stratified_sample_by_length_and_frequency(
        train_words, word_frequencies, NUM_STRATIFIED_SAMPLES)

    for word in sampled_words:
        # For each word, generate initial masked states
        initial_masked_states = process_word(word)

        for initial_state in initial_masked_states:
            game_states = []

            # Simulate the game for the initial masked state
            won, final_word, attempts = play_game_with_a_word(
                model, initial_state, char_frequency, max_word_length, 
                device, max_attempts=6, normalize=True)

            # Record the game states
            # You might need to modify 'play_game_with_a_word' to return all states
            game_states.append((won, final_word, attempts))

            # Save the game states for this initial state in a .pkl file
            file_path = pkls_dir / f"{word}_from_{initial_state}.pkl"
            with open(file_path, 'wb') as file:
                pickle.dump(game_states, file)

    # Update the list and iteration counter
    train_words = [word for word in train_words if word not in sampled_words]
    iteration += 1


In [ ]:
STOP

##### Data Reading and Feature Engineering

In [ ]:
from scr.feature_engineering import build_feature_set, \
    process_single_word_inference, calculate_char_frequencies
from scr.utils import *

import random
from collections import Counter
from scr.utils import read_words
import gc

# MASK_PROB = 0.5

# # Define the base directory for the dataset
# base_dataset_dir = Path('data/')

# Limit the number of words to a smaller number for debugging
train_words = read_words(base_dataset_dir / 'train_words.txt', limit=None)
test_words = read_words(base_dataset_dir / 'test_words.txt', limit=None)
val_words = read_words(base_dataset_dir / 'val_words.txt', limit=None)

gc.collect()

# word_list = word_list[:100]
# # # Randomly select 1000 words
# # unseen_words = random.sample(word_list, 1000)c

# def calculate_word_frequencies(word_list):
#     word_counts = Counter(word_list)
#     total_words = sum(word_counts.values())
#     return {word: count / total_words for word, count in word_counts.items()}

# import random

# # ..
# word_frequencies = calculate_word_frequencies(word_list)
char_frequency = calculate_char_frequencies(train_words)
max_word_length = max(len(word) for word in train_words)

print(base_dataset_dir)

##### Testing Single Word inferece Pipeline

In [ ]:
# # For inference
from scr.feature_engineering import process_single_word_inference
NGRAM_N = 3
# Example usage
inference_word = "_p_"
inference_features, inference_missed_chars = \
    process_single_word_inference(inference_word, \
        char_frequency, max_word_length, ngram_n=NGRAM_N)

In [ ]:
inference_features.shape

##### Model Building

In [ ]:
# In your main script or Jupyter Notebook
from scr.model import RNN
from scr.feature_engineering import process_single_word_inference, \
    char_to_idx, idx_to_char, calculate_char_frequencies, \
        get_missed_characters

# Configuration for the RNN model
# Configuration for the RNN model
config = {
    'rnn': 'LSTM',
    'vocab_size': 27,  # Assuming 26 letters + 1 for underscore
    'hidden_dim': 256,
    'num_layers': 20,
    'embedding_dim': 200,
    'output_mid_features': 200,
    'miss_linear_dim': 50,
    'dropout': 0.5, 
    'use_embedding': True,
    'lr': .00001, 
    'input_feature_size': 3, # Number of features excluding the embedding dimension
    'step_size': 15, 
    'gamma': 0.341,
    'use_cuda': True  # Set to True to use CUDA, False to use CPU
}


In [ ]:
from scr.game import guess_character
from scr.game import play_game_with_a_word

# Initialize RNN model
model = RNN(config)
model = model.to(device)

model.save_model('models/model.pth') # HangmanAPI will load it from here
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##### Playing Games

In [ ]:
words = ['a p p l e ', 'kale', '___']
set_seed(42)

# Modelin interacting with my/dummy api
for word in words:
    won, final_word, attempts = play_game_with_a_word(
        model, word, char_frequency, \
        max_word_length, device, max_attempts=6, normalize=True)

    print(won, final_word, attempts)

##### Training Pipeline

In [ ]:
from torch.nn.utils.rnn import pad_sequence
import torch
from torch.utils.data import DataLoader

def train_on_data_loader(model, data_loader, device=device):
    total_actual_penalty = 0
    total_miss_penalty = 0
    total_batches = 0

    model.train()
    model.to(device)

    for i, batch in enumerate(data_loader):
        inputs, labels, miss_chars, lengths, original_words = batch

        inputs = inputs.to(device)
        labels = labels.to(device)
        miss_chars = miss_chars.to(device)
        # lengths = lengths.to(device)  # Uncomment this if lengths need to be on the device

        outputs = model(inputs, lengths, miss_chars)
       
        actual_penalty, miss_penalty = model.calculate_loss(outputs, labels, \
            lengths, miss_chars, vocab_size=27)

    
        total_actual_penalty += actual_penalty.item()
        total_miss_penalty += miss_penalty.item()
        total_batches += 1

        model.optimizer.zero_grad()
        actual_penalty.backward()
        model.optimizer.step()

    avg_actual_penalty = total_actual_penalty / total_batches if total_batches > 0 else 0
    avg_miss_penalty = total_miss_penalty / total_batches if total_batches > 0 else 0
    return avg_actual_penalty, avg_miss_penalty


    avg_actual_penalty = total_actual_penalty / total_batches if total_batches > 0 else 0
    avg_miss_penalty = total_miss_penalty / total_batches if total_batches > 0 else 0
    return avg_actual_penalty, avg_miss_penalty

In [ ]:
from collections import defaultdict

from collections import defaultdict

def validate_model_on_dataset(model, val_words, char_frequency, \
    max_word_length, device, max_attempts=6, normalize=True, \
        number_of_games=1000):
    total_wins, total_attempts, total_games = 0, 0, 0
    word_stats = {}
    word_length_stats = defaultdict(lambda: {"wins": 0, "losses": 0, "total_attempts": 0, "games": 0})

    for word in val_words:
        if total_games >= number_of_games:
            break

        word_length = len(word)
        masked_word = "_" * word_length
        won, final_word, attempts = play_game_with_a_word(model, word, \
            char_frequency, max_word_length, device, max_attempts, normalize)

        # Update word-specific stats
        word_stats[word] = {
            "won": won,
            "final_word": final_word,
            "attempts_used": attempts
        }

        # Update word length-specific stats
        word_length_stats[word_length]["games"] += 1
        word_length_stats[word_length]["total_attempts"] += attempts
        if won:
            word_length_stats[word_length]["wins"] += 1
        else:
            word_length_stats[word_length]["losses"] += 1

        # Update overall stats
        total_wins += int(won)
        total_attempts += attempts
        total_games += 1

    win_rate = total_wins / total_games if total_games > 0 else 0
    avg_attempts = total_attempts / total_games if total_games > 0 else 0

    # Calculating detailed statistics for each word length
    for length, stats in word_length_stats.items():
        stats["win_rate"] = stats["wins"] / stats["games"]
        stats["avg_attempts"] = stats["total_attempts"] / stats["games"]

    return {
        "win_rate": win_rate,
        "average_attempts": avg_attempts,
        "total_games": total_games,
        "total_wins": total_wins,
        "total_losses": total_games - total_wins,
        "word_stats": word_stats,
        "word_length_stats": dict(word_length_stats)  # Convert defaultdict to dict for readability
    }



In [ ]:
from collections import defaultdict

def calculate_character_metrics(final_word, original_word):
    true_positives, false_positives, false_negatives = defaultdict(int), \
        defaultdict(int), defaultdict(int)

    for char in set(original_word):
        if char in final_word:  # If the char is in the guessed word
            if char in original_word:  # If the char is also in the original word
                true_positives[char] += 1
            else:
                false_positives[char] += 1
        else:
            if char in original_word:  # If the char is in the original word but not in the guess
                false_negatives[char] += 1

    return true_positives, false_positives, false_negatives



def validate_model_on_game(model, val_words, char_frequency, max_word_length, \
    device, max_attempts=6, normalize=True, number_of_games=1000):
    total_wins, total_attempts, total_games = 0, 0, 0
    word_length_stats = defaultdict(lambda: {"wins": 0, "losses": 0, \
            "total_attempts": 0, "games": 0})
    word_stats = {}

    for word in val_words:
        if total_games >= number_of_games:
            break

        word_length = len(word)
        won, final_word, attempts = play_game_with_a_word(model, word, \
            char_frequency, max_word_length, device, max_attempts, normalize)

        # Update word-specific stats
        word_stats[word] = {
            "won": won,
            "final_word": final_word,
            "attempts_used": attempts
        }

        # Update word length-specific stats
        word_length_stats[word_length]["games"] += 1
        word_length_stats[word_length]["total_attempts"] += attempts
        
        if won:
            word_length_stats[word_length]["wins"] += 1
        else:
            word_length_stats[word_length]["losses"] += 1

        # Update overall stats
        total_wins += int(won)
        total_attempts += attempts
        total_games += 1

    win_rate = total_wins / total_games if total_games > 0 else 0
    avg_attempts = total_attempts / total_games if total_games > 0 else 0

    # Calculating detailed statistics for each word length
    for length, stats in word_length_stats.items():
        stats["win_rate"] = stats["wins"] / stats["games"]
        stats["avg_attempts"] = stats["total_attempts"] / stats["games"]

    return {
        "win_rate": win_rate,
        "average_attempts": avg_attempts,
        "total_games": total_games,
        "total_wins": total_wins,
        "total_losses": total_games - total_wins,
        "word_stats": word_stats,
        "word_length_stats": dict(word_length_stats)  # Convert defaultdict to dict for readability
    }


##### Performence testing of Untrain Model

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path
from scr.plot_utils import plot_word_stats

# validation_on_dataset = validate_model_on_dataset(model, val_words, \
#     char_frequency, max_word_length, device, max_attempts=6, \
#         normalize=True, number_of_games=1000)

# # Example usage
# # epoch_number = 1
# save_path = 'plots/untrain_model_stats.png'  # Example custom path
# plot_word_stats(validation_on_dataset['word_length_stats'], save_path)

##### Train

In [ ]:
for epoch in tqdm(range(num_epochs), desc='Epochs'):
    epoch_performance_metrics = {}
    total_actual_penalty, total_miss_penalty = 0, 0
    total_batches = 0  # Counter for the total number of batches

    for pkl_file in tqdm(pkl_files, desc='PKL Files', leave=False):
        try:
            with open(pkl_file, 'rb') as file:
                train_dataset = pickle.load(file)

            sampler = PerformanceBasedSampler(train_dataset, initial_performance_metrics)
            train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, collate_fn=collate_fn)

            for batch in train_loader:
                avg_actual_penalty, avg_miss_penalty = train_on_batch(model, batch, device)
                total_actual_penalty += avg_actual_penalty
                total_miss_penalty += avg_miss_penalty
                total_batches += 1  # Increment the batch counter

            # Aggregate metrics for the current .pkl file
            pkl_file_name = pkl_file.stem
            epoch_performance_metrics[pkl_file_name] = {
                'avg_actual_penalty': avg_actual_penalty,
                'avg_miss_penalty': avg_miss_penalty
            }

        except Exception as e:
            tqdm.write(f"Error processing {pkl_file.stem}: {e}")

    # Calculate average penalties for the entire epoch
    avg_epoch_actual_penalty = total_actual_penalty / total_batches if total_batches > 0 else 0
    avg_epoch_miss_penalty = total_miss_penalty / total_batches if total_batches > 0 else 0

    # Validation step and other epoch-wise processing
    # ...

    all_epochs_performance_metrics[epoch] = epoch_performance_metrics

# Continue with validation, scheduler update, and other steps


In [ ]:
import torch
from torch.utils.data import DataLoader
from pathlib import Path
import pickle
import gc
from scr.utils import load_dataset_pickle
from scr.custom_sampler import PerformanceBasedSampler, update_sampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from scr.dataset import  HangmanDataset, collate_fn
from tqdm import tqdm
import time

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Assuming the existence of necessary functions, model setup, and collate_fn

num_epochs = 20
batch_size = 64
shuffle = True

initial_performance_metrics = {}

# Initialize the scheduler for the optimizer
scheduler = ReduceLROnPlateau(model.optimizer, mode='max', factor=0.5, \
    patience=2, verbose=True)

# # Define the base directory for the dataset
# base_dataset_dir = Path('dataset/pkl')


# # Debugging: Print the directory and number of .pkl files found
# print(f"Dataset directory: {base_dataset_dir}")
# print(f"Number of .pkl files found: {len(pkl_files)}")

# Dictionary to store performance metrics for each epoch and .pkl file
all_epochs_performance_metrics = {}

for epoch in tqdm(range(num_epochs), desc='Epochs'):
    # # print(f'Epoch {epoch+1}:')
    epoch_performance_metrics = {}
    # print(epoch_performance_metrics)

    # print(initial_performance_metrics)

    total_actual_penalty, total_miss_penalty = 0, 0

    for pkl_file in tqdm(pkl_files, desc='PKL Files', leave=False):
        batch_performence_etriccs = ()
        try:
            start_time = time.time()
            # Load the dataset from the .pkl file
            with open(pkl_file, 'rb') as file:
                train_dataset = pickle.load(file)

            # # Initialize sampler for each .pkl file
            # print('sampler')
            sampler = PerformanceBasedSampler(train_dataset, \
                initial_performance_metrics)

            # # Initialize DataLoader with the sampler
            # print('train')
            train_loader = DataLoader(train_dataset, \
                batch_size=batch_size, sampler=sampler, collate_fn=collate_fn)

            # Training step
            avg_actual_penalty, avg_miss_penalty = train_on_batch(model, \
                train_loader, device)

            # Accumulate penalties
            total_actual_penalty += avg_actual_penalty
            total_miss_penalty += avg_miss_penalty

            # Debug lines to print out the penalties for each .pkl file
            # print(f"File: {pkl_file.stem} - Epoch {epoch+1}: Avg Actual Penalty: {avg_actual_penalty}, Avg Miss Penalty: {avg_miss_penalty}")

            # Update metrics for the current .pkl file
            pkl_file_name = pkl_file.stem
            
            batch_performance_metrics[pkl_file_name] = {
                'avg_actual_penalty': avg_actual_penalty,
                'avg_miss_penalty': avg_miss_penalty
            }

            end_time = time.time()
            processing_time = end_time - start_time
            # tqdm.write(f"File: {pkl_file.stem} - Epoch {epoch+1}: Processed in {processing_time:.2f} seconds")

        except Exception as e:
            tqdm.write(f"Error processing {pkl_file.stem}: {e}")

    

    # Calculate average penalties for the epoch
    avg_epoch_actual_penalty = total_actual_penalty / len(pkl_files)
    avg_epoch_miss_penalty = total_miss_penalty / len(pkl_files)

    # Validation step
    # Store metrics for the epoch
    all_epochs_performance_metrics[epoch] = epoch_performance_metrics

    validation_results = validate_model_on_dataset(model, val_words, \
        char_frequency, max_word_length, device, max_attempts=6, normalize=True, \
            number_of_games=10000)

    plot_word_stats(validation_results['word_length_stats'], epoch)
    
    # Print epoch summary
    print(f"Epoch {epoch+1}: Avg Actual Penalty: {avg_epoch_actual_penalty}, Avg Miss Penalty: {avg_epoch_miss_penalty}")

    print(f"Epoch {epoch+1}: Validation - Win Rate: {validation_results['win_rate']}")

    # Update learning rate
    scheduler.step(validation_results['win_rate'])

    # Update initial_performance_metrics for the next epoch
    initial_performance_metrics.update(validation_results['word_length_stats'])

    print(initial_performance_metrics)

    _ = gc.collect()

print('Training completed')

In [ ]:
STOP

##### Tuning

In [ ]:
import torch
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
from scr.model import RNN
from torch.optim.lr_scheduler import StepLR
from scr.utils import *
from scr.early_stopping import EarlyStopping
import optuna

N_GAMES_PER_EPOCH = 10 ** 3
MAX_ATTEMPTS = 6
NUM_EPOCHS = 20

def objective(trial, dataset, static_config, num_epochs):
    dynamic_config = optuna_dynamic_hyperparameters(trial)
    config = {**static_config, **dynamic_config}
    # print(f"Trial {trial.number}: Configuration - {config}")

    kfold = KFold(n_splits=5, shuffle=True)
    aggregated_metrics = []

    best_objective_value = float('-inf')
    best_model_state = None

    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        # print(f"Starting Fold {fold + 1}")
        train_loader, val_loader = get_data_loaders(dataset, train_idx, val_idx)
        model, optimizer = initialize_model(config)

        scheduler = StepLR(optimizer, step_size=trial.suggest_int("step_size", 5, 20), \
            gamma=trial.suggest_float("gamma", 0.1, 0.5))
        
        early_stopping = EarlyStopping(patience=10, delta=0.001)  # Early stopping instance


        for epoch in range(num_epochs):
            _, _ = train_one_epoch(model, train_loader)

            scheduler.step()
            win_rate, average_attempts, win_rate_by_length = simulate_games(
                    model=model,
                    word_list=None,  # Not using a word list in this scenario
                    char_to_idx=char_to_idx, 
                    idx_to_char=idx_to_char,
                    char_frequency=char_frequency, 
                    max_word_length=max_word_length, 
                    device=device, 
                    val_loader=val_loader,  # Pass the validation DataLoader
                    max_games_per_epoch=N_GAMES_PER_EPOCH,  # Number of games for validation
                    normalize=True, 
                    max_attempts=MAX_ATTEMPTS
            
            )

            for epoch in range(num_epochs):
    # Training step
    avg_actual_penalty, avg_miss_penalty = train_one_epoch(model, train_loader, device)

    print(f"Epoch {epoch+1}: Training data - Avg Actual Penalty: {avg_actual_penalty}, Avg Miss Penalty: {avg_miss_penalty}")

    # Validation step

    # win_rate, average_attempts, win_rate_by_length \
    #     = simulate_hangman_games(model, char_to_idx, idx_to_char, \
    #         char_frequency, max_word_length, device, word_list=None, \
    #     val_loader=test_loader, num_games=1000, max_attempts=6, normalize=True)

    
    avg_actual_penalty, avg_miss_penalty = train_one_epoch(model, train_loader, device)


    win_rate, average_attempts, win_rate_by_length \
        = simulate_hangman_games(model, char_to_idx, idx_to_char, \
            char_frequency, max_word_length, device, word_list=None, \
        val_loader=test_loader, num_games=1000, max_attempts=6, normalize=True)

    #         win_rate, average_attempts, win_rate_by_length = validate_one_epoch(model, \
    #             val_loader, char_to_idx, idx_to_char, char_frequency, max_word_length, device)

            aggregated_metrics.append((win_rate, average_attempts, win_rate_by_length))

            objective_value = calculate_objective_value(win_rate, \
                average_attempts, win_rate_by_length)
            
            trial.report(objective_value, epoch)

            if objective_value > best_objective_value:
                best_objective_value = objective_value
                best_model_state = model.state_dict()  # Update best model state

            # Check early stopping
            if early_stopping(objective_value):
                break  # Stop the current fold if early stopping criteria are met

            # Pruning check
            if trial.should_prune():
                return process_aggregated_metrics(aggregated_metrics)

    # Save the best model at the end of the trial
    if best_model_state:
        model.load_state_dict(best_model_state)
        model.save_model()  # Save the model using your custom method

    final_objective_value = process_aggregated_metrics(aggregated_metrics)
    
    return final_objective_value


def calculate_objective_value(win_rate_percentage, avg_attempts_per_game, \
    win_rate_by_word_length):
    # Define weights for each metric
    weight_for_win_rate = 0.7  # Higher weight for win rate
    weight_for_average_attempts = 0.3  # Lower weight for average attempts

    # Normalize win rate (converting percentage to a range of 0 to 1)
    normalized_win_rate = win_rate_percentage / 100

    # Normalize average attempts (assuming max attempts per game is known)
    max_attempts_per_game = 6  # Example: maximum number of attempts allowed
    normalized_avg_attempts = avg_attempts_per_game / max_attempts_per_game

    # Normalize win rate by word length
    normalized_win_rate_by_length = normalize_win_rate_by_length(win_rate_by_word_length)

    # Combine metrics into a single objective value
    objective_value = (weight_for_win_rate * normalized_win_rate) - \
                      (weight_for_average_attempts * normalized_avg_attempts) + \
                      normalized_win_rate_by_length
    return objective_value

def normalize_win_rate_by_length(win_rate_by_length_dict):
    # Example logic for normalization
    max_possible_win_rate = 1  # Maximum win rate (100% as a decimal)
    normalized_scores = {length: win_rate / max_possible_win_rate for \
        length, win_rate in win_rate_by_length_dict.items()}
    # Average the normalized win rates across all word lengths
    average_normalized_score = sum(normalized_scores.values()) / len(normalized_scores)
    return average_normalized_score

def process_aggregated_metrics(aggregated_metrics):
    # Initialize sums for each metric
    sum_win_rate = 0
    sum_average_attempts = 0
    sum_win_rate_by_length = {}

    # Process each set of metrics
    for win_rate, average_attempts, win_rate_by_length in aggregated_metrics:
        sum_win_rate += win_rate
        sum_average_attempts += average_attempts
        for length, rate in win_rate_by_length.items():
            sum_win_rate_by_length[length] = sum_win_rate_by_length.get(length, 0) + rate

    # Calculate averages
    num_entries = len(aggregated_metrics)
    avg_win_rate = sum_win_rate / num_entries
    avg_average_attempts = sum_average_attempts / num_entries
    avg_win_rate_by_length = {length: rate / num_entries for length, \
        rate in sum_win_rate_by_length.items()}

    # Now calculate the final objective value using these averages
    final_objective_value = calculate_objective_value(avg_win_rate, \
        avg_average_attempts, avg_win_rate_by_length)
    return final_objective_value


# Utility functions (for cleaner code)
def get_data_loaders(dataset, train_idx, val_idx):
    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)
    train_loader = DataLoader(train_subset, batch_size=32, \
        shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_subset, batch_size=32, \
        shuffle=False, collate_fn=collate_fn)
    return train_loader, val_loader

def initialize_model(config):
    model = RNN(config)
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    return model, optimizer

# Utility functions
def optuna_dynamic_hyperparameters(trial):
    # Define and return dynamic hyperparameters based on the trial
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    hidden_dim = trial.suggest_categorical('hidden_dim', [128, 256, 512])
    embedding_dim = trial.suggest_categorical('embedding_dim', [50, 100, 150])
    output_mid_features = trial.suggest_categorical('output_mid_features', [50, 100, 200])
    miss_linear_dim = trial.suggest_categorical('miss_linear_dim', [50, 100, 150])
    dropout = trial.suggest_float('dropout', 0.2, 0.5)
    num_layers = trial.suggest_int('num_layers', 1, 3)

    return {
        'lr': lr,
        'hidden_dim': hidden_dim,
        'embedding_dim': embedding_dim,
        'output_mid_features': output_mid_features,
        'miss_linear_dim': miss_linear_dim,
        'dropout': dropout,
        'num_layers': num_layers
    }

In [ ]:
from pathlib import Path

# Define your models directory path
models_dir = Path('models/notebook_2')

# Create the directory if it does not exist
models_dir.mkdir(parents=True, exist_ok=True)

# # Assuming missed_chars_tensor is a list of tensors for missed characters
# dataset = HangmanDataset(features, \
#     labels, missed_chars, original_words)

# Static configuration
static_config = {
    'rnn': 'LSTM',
    'vocab_size': 27,  # 26 English alphabets + 1 (e.g., for underscore)
    'use_embedding': True,  # Typically a design choice
    'input_feature_size': 5,  # Based on your feature engineering strategy
    'models': str(models_dir)
}

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define num_epochs for training in each fold
num_epochs = 4 # Adjust as needed
N_TRIAL = 3

# ====================================================== #
# Define the direction for the objective: 'maximize' or 'minimize'
direction = 'maximize'  # Adjust based on your objective function

study = optuna.create_study(direction=direction)
study.optimize(lambda trial: objective(trial, dataset, \
    static_config, num_epochs), n_trials=N_TRIAL)
# ====================================================== #

In [ ]:
best_params = study.best_params
# Output the best hyperparameters
print("Best trial:")
trial = study.best_trial
print(f"Value: {trial.value}")
print("Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

In [ ]:
best_trial = study.best_trial
print(f"Best trial number: {best_trial.number}")

# Assuming model saving includes the trial number in the filename
best_model_filename = f"models/trial_{best_trial.number+1}/LSTM.pth"
best_model = RNN.load_model(RNN, best_model_filename, device)

In [ ]:
STOP

In [ ]:
# Merge the two dictionaries to form a single configuration
config = {**static_config, **best_params}

config

In [ ]:
# Merge the two dictionaries to form a single configuration
config = {**static_config, **best_params}

# Initialize the RNN model with the combined configuration
model = RNN(config)

model = model.to(device)

# model.load_model(config['LSTM'], 2, 256, trial_number=10)

In [ ]:
STOP

##### Testing on unknown data

In [ ]:
%%capture

from scr.game import predict_next_character, simulate_game
from scr.model import RNN
import random
# random.seed(400)
# Your existing code for initializing the model, etc.

import random

def play_multiple_games(model, num_games, word_list, \
    char_to_idx, idx_to_char, char_frequency, max_word_length, device):
    game_results = []
    
    sampled_words = random.sample(word_list, num_games)  # Select unique words

    for random_word in sampled_words:
        with torch.no_grad():
            won, final_word, attempts_used = simulate_game(
                model, 
                random_word, 
                char_to_idx, 
                idx_to_char, 
                char_frequency, 
                max_word_length, 
                device, 
                normalize=True, 
                max_attempts=6
            )
        game_results.append((won, final_word, attempts_used))

    return game_results

# Example usage
num_games = 10**3
results = play_multiple_games(model, num_games, \
    word_list, char_to_idx, idx_to_char, \
        char_frequency, max_word_length, device)

# Analyzing results
total_wins = sum(result[0] for result in results)
win_rate = (total_wins / num_games) * 100
